# Pending Task: Fix VQQNN model

In [1]:

#Source and credit: https://github.com/MSKX/tensorflow-quantum_dense
#Confirm successful import of packages
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as plt_dates

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from QuantumDense import VQNNModel

import matplotlib.dates as plt_dates

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from QuantumDense import VQNNModel

C:\Users\Spark\AppData\Local\Temp\ipykernel_24364\3669405310.py:17: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  from QuantumDense import VQNNModel


In [2]:
#Confirm successful import of input data, instead of refinitive.dataplatform.eikon API
import yfinance as yf
import pandas as pd
import datetime as date


now = date.date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')

# read input data from CSV file
df = pd.read_csv('datasets/test.csv')

# convert date column to datetime and set as index
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)
df['HL_DELTA'] = df['HIGH'] - df['LOW']
df['RETURNS'] = df['CLOSE'].pct_change()
df['PRX_MA_ND'] = df['CLOSE'].rolling(window=5).mean()
df['VOLATILITY'] = df['CLOSE'].rolling(window=5).std()
df['TP1_RETURNS'] = df['RETURNS'].shift(-1)

df.dropna(inplace=True)


print(df)

            CLOSE     VOLUME   Open   HIGH    LOW  HL_DELTA   RETURNS  \
DATE                                                                    
2021-12-03  46.90   909543.0  46.66  47.25  46.34      0.91  0.006222   
2021-12-06  48.75   740776.0  47.68  49.15  47.36      1.79  0.039446   
2021-12-07  49.57   920421.0  48.96  50.20  48.81      1.39  0.016821   
2021-12-08  49.82   473157.0  49.40  50.27  49.36      0.91  0.005043   
2021-12-09  49.39  1036663.0  49.55  50.12  49.20      0.92 -0.008631   
...           ...        ...    ...    ...    ...       ...       ...   
2022-11-18  56.00   462336.0  56.26  56.94  55.56      1.38  0.012292   
2022-11-21  55.86   379062.0  55.82  56.56  55.62      0.94 -0.002500   
2022-11-22  56.72   441990.0  56.07  56.88  55.99      0.89  0.015396   
2022-11-23  56.47   333914.0  56.66  56.89  56.16      0.73 -0.004408   
2022-11-25  57.12   283691.0  56.89  57.56  56.56      1.00  0.011511   

            PRX_MA_ND  VOLATILITY  TP1_RETURNS  
D

In [3]:
#Create 3 dimensional Tensor
df_x = df[['RETURNS', 'VOLUME', 'PRX_MA_ND', 'VOLATILITY']]
df_y = df['TP1_RETURNS']
df_x.dropna(inplace=True)
df_x

C:\Users\Spark\AppData\Local\Temp\ipykernel_24364\2085320503.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x.dropna(inplace=True)


,RETURNS,VOLUME,PRX_MA_ND,VOLATILITY
DATE,,,,
2021-12-03,0.006222,909543.0,46.474,0.891813
2021-12-06,0.039446,740776.0,46.706,1.308331
2021-12-07,0.016821,920421.0,47.412,1.740236
2021-12-08,0.005043,473157.0,48.330,1.494774
2021-12-09,-0.008631,1036663.0,48.886,1.178656
...,...,...,...,...
2022-11-18,0.012292,462336.0,56.026,0.609902
2022-11-21,-0.002500,379062.0,55.934,0.588795
2022-11-22,0.015396,441990.0,55.902,0.525471


In [5]:
df_y #check

DATE
2021-12-03    0.039446
2021-12-06    0.016821
2021-12-07    0.005043
2021-12-08   -0.008631
2021-12-09    0.028953
                ...   
2022-11-18   -0.002500
2022-11-21    0.015396
2022-11-22   -0.004408
2022-11-23    0.011511
2022-11-25   -0.016632
Name: TP1_RETURNS, Length: 247, dtype: float64

In [6]:
df_x_scaler = MinMaxScaler().fit(df_x)

forward_test_date = '2022-10-01'

fdf_x = df_x.loc[forward_test_date:]
fdf_y = df_y.loc[forward_test_date:]
df_x = df_x.loc[:forward_test_date]
df_y = df_y.loc[:forward_test_date]

fdf_prx = df.loc[forward_test_date:]['CLOSE']
fdf_y_len = len(fdf_y)

df_x_scaled = pd.DataFrame(df_x_scaler.transform(df_x))
fdf_x_scaled = pd.DataFrame(df_x_scaler.transform(fdf_x))

x_train, x_test, y_train, y_test = train_test_split(df_x_scaled,
                                                    df_y,
                                                    test_size=0.25,
                                                    random_state=42)

x_train

,0,1,2,3
29,0.754539,0.133202,0.553488,0.204322
19,0.531912,0.051012,0.607893,0.170471
55,0.519906,0.297936,0.640592,0.251668
93,0.676507,0.031892,0.985624,0.281851
181,0.695942,0.211450,0.551233,0.317577
...,...,...,...,...
106,0.438236,0.058197,0.713319,0.265963
14,0.669497,0.094454,0.466667,0.169921
92,0.318661,0.075631,0.986610,0.279157
179,0.439519,0.095410,0.618041,0.251853


In [7]:
x_train = np.expand_dims(x_train.values, 1).astype(np.float32)
y_train = np.expand_dims(y_train.values, 1).astype(np.float32)
x_validation = np.expand_dims(x_test.values, 1).astype(np.float32)
y_validation = np.expand_dims(y_test.values, 1).astype(np.float32)

In [8]:
qnn_model = VQNNModel()
qnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                     beta_1=0.9,
                                                     beta_2=0.999,
                                                     epsilon=1e-07),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=["mean_squared_error"])
qnn_model.run_eagerly = True

In [11]:
log_dir = "logs\\model\\eikon\\"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
qnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                     beta_1=0.9,
                                                     beta_2=0.999,
                                                     epsilon=1e-07),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=["mean_squared_error"])
qnn_model.run_eagerly = True
qnn_model.fit(x_train, y_train, epochs=1, batch_size=1, callbacks=[tensorboard_callback])

AttributeError: Exception encountered when calling layer 'quantum_layer' (type QuantumLayer).

'TypeError' object has no attribute 'message'

Call arguments received by layer 'quantum_layer' (type QuantumLayer):
  • inputs=tf.Tensor(shape=(1, 1, 3), dtype=float32)

NameError: name 'cirq' is not defined